In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://geodes.santepubliquefrance.fr/#view=map2&c=indicator'
sleep_for = 0.5

In [2]:

def download_current_indicators(driver: webdriver.Chrome):

    x_path_tabular = '//*[@id="tm_table"]'
    x_path_action = '/html/body/main/section[2]/div/div/div[2]/div[1]/a'
    x_path_export = '/html/body/main/section[2]/div/div/div[2]/div[1]/div[2]/ul/li[2]/a'
    x_path_csv = '/html/body/section[1]/div/div/section/div[1]/label[3]'
    x_path_dowload = '/html/body/section[1]/div/div/section/button'
    x_path_map = '//*[@id="tm_map"]'

    driver.find_element(By.XPATH, x_path_tabular).click()
    time.sleep(sleep_for)
    driver.find_element(By.XPATH, x_path_action).click()
    time.sleep(sleep_for)
    driver.find_element(By.XPATH, x_path_export).click()
    time.sleep(sleep_for)
    driver.find_element(By.XPATH, x_path_csv).click()
    time.sleep(sleep_for)
    driver.find_element(By.XPATH, x_path_dowload).click()
    time.sleep(sleep_for)
    driver.find_element(By.XPATH, x_path_map).click()
    time.sleep(sleep_for)

    

In [3]:
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(1.5)


# Click on the search bar and search for 'urgences' to filter the indicators
x_path_search_bar = '/html/body/main[1]/section[1]/div[2]/div[3]/span[1]/input'
search_bar = driver.find_element(By.XPATH, x_path_search_bar)
search_bar.send_keys('urgences')
search_bar.send_keys(Keys.RETURN)

time.sleep(sleep_for)

# Click each indicator

x_path_indic_list = '/html/body/main/section[1]/div[2]/div[4]/div/div'
css_selector_indic_list = 'div.mts:nth-child(1)'

x_path_week_selector = '/html/body/main[2]/section[2]/div/div/section/div/div/div/div/header/div[1]/p[5]/a'
x_path_time_series = '//*[@id="tm_serie1"]'
x_path_remove_all = '/html/body/main[2]/section[1]/div[3]/section[1]/div[1]/fieldset/div/span'


x_path_back = '/html/body/main/section[1]/div[2]/div[6]/div/div[1]/a'
# Get main list containing letter categories (A, B, C, D, ...)
indic_list = driver.find_element(By.CLASS_NAME, 'mts')

indics = indic_list.find_elements(By.XPATH, "./div")

# Click on each letter category
print("indics_letters: ", len(indics))

for indic in indics:
    print("letter: ", indic.text)
    indic.click()
    time.sleep(sleep_for)

    # Get the list of indicators categories for the selected letter category
    # Trouver le premier 'ul' dans ce 'div'
    first_ul = indic.find_element(By.TAG_NAME, "ul")

    # Récupérer directement les 'li' enfants de ce 'ul' (niveau supérieur uniquement)
    list_sub_indic = first_ul.find_elements(By.XPATH, "./li")

    print("list_sub_indic: ", len(list_sub_indic))
    # Click on each indicator category
    for sub_indic in list_sub_indic:
        print("     disease: ", sub_indic.text)
        sub_indic.click()
        time.sleep(sleep_for)

        # Optional second sub indic category
        
        list_second_sub_indic = sub_indic.find_elements(By.TAG_NAME, 'li')
        print("     list_second_sub_indic: ", len(list_second_sub_indic))
        if len(list_second_sub_indic) == 0:
            list_second_sub_indic = [None]
        for second_sub_indic in list_second_sub_indic:
            if second_sub_indic is not None:
                print("         second sub indic: ", second_sub_indic.text)
                second_sub_indic.click()
                time.sleep(sleep_for)

            # Select the main list of indicators
            indic_main = driver.find_element(By.CLASS_NAME, "ui-indic-list")

            # Get the list of indicators for the selected indicator of selected indicator category of the selected letter category
            list_sub_sub_indic = indic_main.find_elements(By.CLASS_NAME, 'ui-collapsible')
            i = 0
            while i < len(list_sub_sub_indic):
                print("         indic_name: ", list_sub_sub_indic[i].text)
                list_sub_sub_indic[i].click()
                time.sleep(sleep_for)

                # Get the list of indicators by age for the selected indicator of selected indicator category of the selected letter category
                try:
                    list_subsubsub_indic = list_sub_sub_indic[i].find_elements(By.TAG_NAME, 'li')
                except:
                    print("             No subsubsub indicator")
                    # Télécharger l'indicateur pour toute les semaines
                    # Cliquer sur le selecteur de semaine
                    week_selector = driver.find_element(By.CLASS_NAME, 'serie-dropdown')
                    week_selector.click()
                    time.sleep(sleep_for)

                    # Cliquer sur chaques semaines
                    time_serie = driver.find_element(By.XPATH, x_path_time_series)
                    weeks = time_serie.find_elements(By.TAG_NAME, 'li')
                    j = 0
                    while j < len(weeks):
                        print("                 week: ", weeks[j].text)
                        weeks[j].click()
                        time.sleep(sleep_for)
                        # Télécharger l'indicateur
                        download_current_indicators(driver)
                        # Cliquer sur le selecteur de semaine
                        week_selector = driver.find_element(By.CLASS_NAME, 'serie-dropdown')
                        week_selector.click()
                        time.sleep(sleep_for)
                        time_serie = driver.find_element(By.XPATH, x_path_time_series)
                        weeks = time_serie.find_elements(By.TAG_NAME, 'li')[1:]
                        j += 1
                    
                    driver.find_element(By.XPATH, x_path_remove_all).click()
                    continue

                finally:
                    list_sub_sub_indic = indic_main.find_elements(By.CLASS_NAME, 'ui-collapsible')
                    i += 1

                for sub_subsub_indic in list_subsubsub_indic:
                    print("             indic: ", sub_subsub_indic.text)
                    sub_subsub_indic.click()
                    time.sleep(sleep_for)

                    # Télécharger l'indicateur pour toute les semaines

                    # Cliquer sur le selecteur de semaine
                    week_selector = driver.find_element(By.CLASS_NAME, 'serie-dropdown')
                    week_selector.click()
                    time.sleep(sleep_for)

                    # Cliquer sur chaques semaines
                    time_serie = driver.find_element(By.XPATH, x_path_time_series)
                    weeks = time_serie.find_elements(By.TAG_NAME, 'li')[1:]
                    j = 101
                    while j < len(weeks):
                        print("                 week: ", weeks[j].text)
                        weeks[j].click()
                        time.sleep(sleep_for)
                        # Télécharger l'indicateur
                        download_current_indicators(driver)
                        # Cliquer sur le selecteur de semaine
                        week_selector = driver.find_element(By.CLASS_NAME, 'serie-dropdown')
                        week_selector.click()
                        time.sleep(sleep_for)
                        time_serie = driver.find_element(By.XPATH, x_path_time_series)
                        weeks = time_serie.find_elements(By.TAG_NAME, 'li')[1:]
                        j += 1

                    # Clear the selected indicators
                    driver.find_element(By.XPATH, x_path_remove_all).click()

            # Remonter l'arboréscence pour revenir à la liste des letter categories et continuer le processus pour chaque indicator category de la lettre selectionnée

            back = driver.find_element(By.XPATH, x_path_back)
            back.click()
            time.sleep(sleep_for)


driver.close()


indics_letters:  8
letter:  A
4
list_sub_indic:  2
     disease:  Allergie
2
     list_second_sub_indic:  0
         indic_name:  Taux de passages aux urgences pour allergie (2024-S28...2010-S01)
             indic:  tous âges
                 week:  2022-S31
                 week:  2022-S30
                 week:  2022-S29
                 week:  2022-S28
                 week:  2022-S27
                 week:  2022-S26
                 week:  2022-S25
                 week:  2022-S24
                 week:  2022-S23
                 week:  2022-S22
                 week:  2022-S21
                 week:  2022-S20
                 week:  2022-S19


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/section[1]/div/div/section/div[1]/label[3]"}
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5d6bf2a0cc5a <unknown>
#1 0x5d6bf26efe2c <unknown>
#2 0x5d6bf273c661 <unknown>
#3 0x5d6bf273c751 <unknown>
#4 0x5d6bf2780f64 <unknown>
#5 0x5d6bf275f5ed <unknown>
#6 0x5d6bf277e303 <unknown>
#7 0x5d6bf275f363 <unknown>
#8 0x5d6bf272f247 <unknown>
#9 0x5d6bf272fb9e <unknown>
#10 0x5d6bf29d322b <unknown>
#11 0x5d6bf29d72d1 <unknown>
#12 0x5d6bf29beade <unknown>
#13 0x5d6bf29d7e32 <unknown>
#14 0x5d6bf29a377f <unknown>
#15 0x5d6bf29fc618 <unknown>
#16 0x5d6bf29fc7f0 <unknown>
#17 0x5d6bf2a0bd8c <unknown>
#18 0x735621694ac3 <unknown>
